## LBC group management


In [299]:
import collections
from dataclasses import dataclass
import pandas as pd
import numpy as np
from collections import namedtuple
group_chars = [c for c in 'ABCDEFGHIJKL']
requirement = namedtuple('requirement', 'target min max')
sc_group_size = requirement(8,6,9)
sc_singles_count = requirement(2,0,4)
sc_couple_count = requirement(3,3,4)
lb_group_size = requirement(6,5,7)
lb_singles_count = requirement(2,0,3)
lb_couple_count = requirement(2,2,3)


### import historical groupings
- columns are groupings, of either type 'lb' or 'sc'
- rows are members
- character denotes membership in a group. same letter same group. null for no involvement

In [300]:
column_names = ['size'] + list(f'{"sc" if i%2 else "lb"}{i//2}'
                             for i in range(29)
                             )
group_history = pd.read_csv("LBC_group_history.csv", names=column_names,header=0)

display(group_history.head(5))

,size,lb0,sc0,lb1,sc1,lb2,sc2,lb3,sc3,lb4,...,sc9,lb10,sc10,lb11,sc11,lb12,sc12,lb13,sc13,lb14
0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,B,NaN,C,NaN,NaN,NaN,NaN,NaN,A,NaN
1,1,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,D,NaN,F,NaN,G,NaN,NaN,B,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,A,E,F,C,I,B,F,C,F,...,A,NaN,A,NaN,NaN,NaN,C,NaN,D,NaN
4,2,NaN,E,E,NaN,NaN,A,NaN,NaN,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E,NaN


### measure group sizes
- get total group sizes using weights from size column


In [301]:
weights = group_history.iloc[:, 0].values

group_sizes = pd.DataFrame([
    [
        np.sum([weights[i] if value == group_char else 0 for i, value in enumerate(group_history[column])])
        for group_char in group_chars
    ] for column in group_history.columns[1:]
],
columns=group_chars,
index=[column for column in group_history.columns[1:]],
).replace(0, np.nan)

display(group_sizes.head(5))


,A,B,C,D,E,F,G,H,I,J,K,L
lb0,6.0,4.0,4.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sc0,7.0,6.0,7.0,5.0,4.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN
lb1,5.0,1.0,4.0,4.0,6.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN
sc1,7.0,8.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lb2,5.0,4.0,3.0,5.0,6.0,4.0,6.0,6.0,3.0,NaN,NaN,NaN


### create met_before lookup dict
- key: group history index
- value: list of group history indexes


In [302]:
all_groups = list()
for col_name, grouping in group_history.iteritems():
    for group_char in set(grouping):
        if group_char in group_chars:
            all_groups.append(group_history.index[group_history[col_name] == group_char].tolist())

met_before = {i: set([i,]) for i, groups in group_history.iterrows()}
for group in all_groups:
    for i in group:
        for j in group:
            met_before[i].add(j)



### data sanity check
- expect 10-25 unique groups in each round
- groups around 4-8 in size
- max possible group memberships is 29
- 20-50% of members never active before


In [303]:
# filter row data
# [list(filter(lambda x: x>0, grouping)) for i, grouping in group_sizes.iterrows()]

group_counts = [np.sum(len(list(filter(lambda x: x>0, grouping)))) for i, grouping in group_sizes.iterrows()]

features = pd.DataFrame(
    [
        [
            # number of groups
            np.mean(group_counts),
            np.min(group_counts),
            np.max(group_counts),
        ],
        [
            # group size
            np.mean([np.nanmean(group_sizes)]),
            np.min([np.nanmin(group_sizes)]),
            np.max([np.nanmax(group_sizes)]),
        ],
        [
            # socializing
            np.mean([len(met_set) for i, met_set in met_before.items()]),
            np.min([len(met_set) for i, met_set in met_before.items()]),
            np.max([len(met_set) for i, met_set in met_before.items()]),
        ],
    ],
    columns=['mean', 'min', 'max'],
    index=['number of groups', 'group size head count', 'met counts'],
)

display(features)

,mean,min,max
number of groups,4.448276,0.0,11.0
group size head count,5.906977,1.0,10.0
met counts,11.403226,3.0,46.0


# Create next SC and LB groupings
- confirm group requirements


In [304]:
display(pd.DataFrame([
    sc_group_size, sc_singles_count, sc_couple_count,
    lb_group_size, lb_singles_count, lb_couple_count,
],
    columns=['target', 'min', 'max'],
    index=['sc group size', 'sc singles count', 'sc couple count',
           'lb group size', 'lb singles count', 'lb couple count',],
))

,target,min,max
sc group size,8,6,9
sc singles count,2,0,4
sc couple count,3,3,4
lb group size,6,5,7
lb singles count,2,0,3
lb couple count,2,2,3


- create groups
- disperse indexes by order of met counts
- check member requirements
- check total size limits

In [305]:
# sort indexes by weight (couples first), then met_before count (most first_
ordered_indexes = collections.OrderedDict(sorted(met_before.items(), key=lambda met_set: (-weights[met_set[0]], -len(met_set[1]))))
# for i, met_set in ordered_indexes.items():
#     print(i, len(met_set))




In [314]:
indexes_list = list(ordered_indexes.keys())

min_sc_groups_count = np.sum(weights) / sc_group_size.target
min_lb_groups_count = np.sum(weights) / lb_group_size.max

@dataclass
class NewGroup:
    members: list
    singles: int
    couples: int
    size: int

lb_groupings = [NewGroup([],0,0,0),]
sc_groupings = [NewGroup([],0,0,0),]

def i_can_join(asking_index: int, new_group: NewGroup, please=False):
    if asking_index in new_group.members:
        raise Exception("already a member of this group!")

    if new_group.size >= lb_group_size.max:
        return False
    if weights[asking_index] == 1 and new_group.singles >= lb_singles_count.max:
        return False
    if weights[asking_index] == 2 and new_group.couples >= lb_couple_count.max:
        return False

    if not please:
        if len(set(new_group.members).intersection(met_before[asking_index])) > 0:
            return False

        if new_group.size >= lb_group_size.target:
            return False
        if weights[asking_index] == 1 and new_group.singles >= lb_singles_count.target:
            return False
        if weights[asking_index] == 2 and new_group.couples >= lb_couple_count.target:
            return False

    if please != "pretty please with cherry on top":
        if len(set(new_group.members).intersection(met_before[asking_index])) > 1:
            return False
    return True

placed_indexes = set()
for index in indexes_list:
    this_index_placed = False
    for group_i, group in enumerate(lb_groupings):

        #  fill an empty group
        if len(group.members) == 0:
            assert(group.size==0)
            lb_groupings[group_i].members.append(index)
            this_index_placed = True
            break

        # create a new group
        if len(lb_groupings) < min_lb_groups_count:
            lb_groupings.append(NewGroup([index,], 0, 0, 0))
            this_index_placed = True
            group_i = len(lb_groupings)-1
            break

        # join group if not full and unmet members
        elif i_can_join(index, group):
            lb_groupings[group_i].members.append(index)
            this_index_placed = True
            break

    # uncomment to allow going beyond targets
    # if not this_index_placed:
    #     for group_i, group in enumerate(lb_groupings):
    #         if i_can_join(index, group, please=True):
    #             lb_groupings[group_i].members.append(index)
    #             this_index_placed = True
    #             break

    if this_index_placed:
        lb_groupings[group_i].size += weights[index]
        if weights[index] == 1:
            lb_groupings[group_i].singles += 1
        else:
            lb_groupings[group_i].couples += 1
        placed_indexes.add(index)



# Results when being strict with targets

In [315]:
print(f"need at least {int(min_lb_groups_count)} groups. and ended up creating {len(lb_groupings)} groups")
display(lb_groupings)


need at least 29 groups. and ended up creating 30 groups


[NewGroup(members=[10, 57, 70, 24], singles=2, couples=2, size=6),
 NewGroup(members=[107, 97, 122, 86], singles=2, couples=2, size=6),
 NewGroup(members=[33, 54, 62, 109], singles=2, couples=2, size=6),
 NewGroup(members=[115, 80, 3, 7], singles=2, couples=2, size=6),
 NewGroup(members=[93, 21, 102, 65], singles=2, couples=2, size=6),
 NewGroup(members=[61, 46, 103, 45], singles=2, couples=2, size=6),
 NewGroup(members=[76, 77, 99, 114], singles=2, couples=2, size=6),
 NewGroup(members=[87, 79, 23, 69], singles=2, couples=2, size=6),
 NewGroup(members=[110, 83, 71, 88], singles=2, couples=2, size=6),
 NewGroup(members=[17, 105, 120, 66], singles=2, couples=2, size=6),
 NewGroup(members=[48, 123, 19, 8], singles=2, couples=2, size=6),
 NewGroup(members=[2, 20, 15, 38], singles=2, couples=2, size=6),
 NewGroup(members=[4, 82, 119, 29], singles=2, couples=2, size=6),
 NewGroup(members=[43, 39, 1, 14], singles=2, couples=2, size=6),
 NewGroup(members=[5, 27, 50, 16], singles=2, couples=2,

# Results when being loose with targets

In [313]:
print(f"need at least {int(min_lb_groups_count)} groups. and ended up creating {len(lb_groupings)} groups")
display(lb_groupings)

need at least 29 groups. and ended up creating 30 groups


[NewGroup(members=[10, 57, 26, 66], singles=1, couples=3, size=7),
 NewGroup(members=[107, 97, 31, 8], singles=1, couples=3, size=7),
 NewGroup(members=[33, 54, 36, 15], singles=1, couples=3, size=7),
 NewGroup(members=[115, 80, 44, 38], singles=1, couples=3, size=7),
 NewGroup(members=[93, 21, 47, 119], singles=1, couples=3, size=7),
 NewGroup(members=[61, 46, 49, 1], singles=1, couples=3, size=7),
 NewGroup(members=[76, 77, 51, 14], singles=1, couples=3, size=7),
 NewGroup(members=[87, 79, 55, 29], singles=1, couples=3, size=7),
 NewGroup(members=[110, 83, 58, 50], singles=1, couples=3, size=7),
 NewGroup(members=[17, 105, 59, 16], singles=1, couples=3, size=7),
 NewGroup(members=[48, 123, 72, 37], singles=1, couples=3, size=7),
 NewGroup(members=[2, 20, 73, 91], singles=1, couples=3, size=7),
 NewGroup(members=[4, 82, 74, 106], singles=1, couples=3, size=7),
 NewGroup(members=[43, 39, 75, 111], singles=1, couples=3, size=7),
 NewGroup(members=[5, 27, 78, 52], singles=1, couples=3, s